In [8]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os

import sys
sys.path.insert(0, '/Users/ageller/VISUALIZATIONS/Firefly')
sys.path.insert(0,'/Users/agurvich/research/repos/firefly/src')
from firefly.data_reader import FIREreader,SimpleFIREreader

from firefly.data_reader import Octree
from abg_python.galaxy.gal_utils import Galaxy
galaxy = Galaxy('m12b_res57000',600)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Couldn't find a metadata file... for
 Metadata object at /Users/agurvich/scratch/data/metal_diffusion/m12b_res57000/metadata/meta_Galaxy_600.hdf5
Tracing the rockstar halo files with fancy:True and None Gyr smoothing.
No rstar 1/2 in halo or metadata files, we will need to calculate it ourselves.


In [15]:
center = galaxy.scom#np.zeros(3)#galaxy.scom+ [-39,-168,189]
reader = FIREreader(
    galaxy.snapdir,
    galaxy.snapnum,
    ptypes=[0,4],
    decimation_factors = [10,10],
    fields = ['Temperature','Masses','AgeGyr'],
    logFlags = [1,0,0],
    clean_JSONdir=True,
    write_startup=True)
reader.loadData(com=center)
reader.settings['showFPS'] = True

JSONdir is None, defaulting to /Users/agurvich/research/repos/firefly/src/firefly/static/data/Data
Loading ptype 4
Make sure each field_array (2) has a field_radius_flag (0), assuming False.
Loading ptype 0
Make sure each field_array (2) has a field_radius_flag (0), assuming False.


In [16]:
reader.settings['color']['PartType0']=[1,0,0,1]
reader.settings['color']['PartType4']=[0,0,1,1]

reader.settings['sizeMult']['PartType0']=1
reader.settings['sizeMult']['PartType4']=1
reader.dumpSettingsToJSON()


('/Users/agurvich/research/repos/firefly/src/firefly/static/data/Data/DataSettings.json',
 '/Users/agurvich/research/repos/firefly/src/firefly/static/data/Data/DataSettings.json')

In [152]:
reader.createOctrees([True,True],npart_min_node=1e4,npart_max_node=1e5)

Bulding octree of 6225729 points
0.00%	0.16%	0.32%	0.48%	0.64%	0.80%	0.96%	1.12%	1.28%	1.45%	1.61%	1.77%	1.93%	2.09%	2.25%	2.41%	2.57%	2.73%	2.89%	3.05%	3.21%	3.37%	3.53%	3.69%	3.85%	4.02%	4.18%	4.34%	4.50%	4.66%	4.82%	4.98%	5.14%	5.30%	5.46%	5.62%	5.78%	5.94%	6.10%	6.26%	6.42%	6.59%	6.75%	6.91%	7.07%	7.23%	7.39%	7.55%	7.71%	7.87%	8.03%	8.19%	8.35%	8.51%	8.67%	8.83%	8.99%	9.16%	9.32%	9.48%	9.64%	9.80%	9.96%	10.12%	10.28%	10.44%	10.60%	10.76%	10.92%	11.08%	11.24%	11.40%	11.56%	11.73%	11.89%	12.05%	12.21%	12.37%	12.53%	12.69%	12.85%	13.01%	13.17%	13.33%	13.49%	13.65%	13.81%	13.97%	14.13%	14.30%	14.46%	14.62%	14.78%	14.94%	15.10%	15.26%	15.42%	15.58%	15.74%	15.90%	16.06%	16.22%	16.38%	16.54%	16.70%	16.87%	17.03%	17.19%	17.35%	17.51%	17.67%	17.83%	17.99%	18.15%	18.31%	18.47%	18.63%	18.79%	18.95%	19.11%	19.27%	19.44%	19.60%	19.76%	19.92%	20.08%	20.24%	20.40%	20.56%	20.72%	20.88%	21.04%	21.20%	21.36%	21.52%	21.68%	21.84%	22.01%	22.17%	22.33%	22.49%	22.65%	22.81%	22.97%	23.13%	23.29%	23.45%	2

In [17]:
reader.max_npart_per_file = 1e5

In [18]:
reader.writeToDisk(symlink=False)

Removing old ffly files from /Users/agurvich/research/repos/firefly/src/firefly/static/data/Data


''

In [45]:
from firefly.data_reader import ArrayReader
# Load your data. Here we're creating random data.
coords = np.random.randn( 20000, 3 )
fields = np.random.random(size=coords[:,0].size)
my_arrayReader = ArrayReader(
    coords,
    fields=fields,
    write_jsons_to_disk=True)

JSONdir is None, defaulting to /Users/agurvich/research/repos/firefly/src/firefly/static/data/Data
Make sure each tracked_array (1) has a tracked_filter_flag (0), assuming True.
Make sure each tracked_array (1) has a tracked_colormap_flag (0), assuming True.
Outputting: PGroup_0 - 20000/20000 particles - 1 tracked fields


In [40]:
octree = reader.particleGroups[0].octree

root = octree.nodes['']
child = octree.nodes['4']

In [29]:
root.center-root.width/2

array([-9468.1  , -8766.78 , -8872.656], dtype=float32)

In [20]:
root.center

array([-1825.021 , -1123.702 , -1229.5784], dtype=float32)

In [33]:
child.fields

array([ 1.00252092e+01,  1.16980360e-02, -1.09850803e+11, -7.46773799e+10,
       -2.86814218e+10])

In [28]:
child.center-child.width/2

array([-9468.09912109, -8766.78015137, -1229.57836914])

In [24]:
import time

In [26]:
init_time = time.time()
node_list = octree.buildOctree()
print()
print(time.time()-init_time,'s elapsed')

0.00%	0.79%	1.59%	2.38%	3.17%	3.97%	4.76%	5.56%	6.35%	7.14%	7.94%	8.73%	9.52%	10.32%	11.11%	11.91%	12.70%	13.49%	14.29%	15.08%	15.87%	16.67%	17.46%	18.25%	19.05%	19.84%	20.64%	21.43%	22.22%	23.02%	23.81%	24.60%	25.40%	26.19%	26.98%	27.78%	28.57%	29.37%	30.16%	30.95%	31.75%	32.54%	33.33%	34.13%	34.92%	35.72%	36.51%	37.30%	38.10%	38.89%	39.68%	40.48%	41.27%	42.06%	42.86%	43.65%	44.45%	45.24%	46.03%	46.83%	47.62%	48.41%	49.21%	50.00%	50.80%	51.59%	52.38%	53.18%	53.97%	54.76%	55.56%	56.35%	57.14%	57.94%	58.73%	59.53%	60.32%	61.11%	61.91%	62.70%	63.49%	64.29%	65.08%	65.87%	66.67%	67.46%	68.26%	69.05%	69.84%	70.64%	71.43%	72.22%	73.02%	73.81%	74.61%	75.40%	76.19%	76.99%	77.78%	78.57%	79.37%	80.16%	80.95%	81.75%	82.54%	83.34%	84.13%	84.92%	85.72%	86.51%	87.30%	88.10%	88.89%	89.69%	90.48%	91.27%	92.07%	92.86%	93.65%	94.45%	95.24%	96.03%	96.83%	97.62%	98.42%	99.21%	
29.118792057037354 s elapsed


In [32]:
octree.nodes['52222050']

OctNode(52222050):2143 points - 4 fields

In [38]:
!cat test.json

[{"2": 1}, {"3": 2}]

In [37]:
import json
arr = [{2:1},{3:2}]
with open('test.json', 'w') as f:
    json.dump(arr, f)

In [43]:
octree.field_names = list(octree.field_names)

In [79]:
np.sum([node.buffer_size for node in foo[0]])

9949

In [67]:
len(sorted_node_list)

5229

In [66]:
break_index

4569

In [71]:
foo[-1]

array([OctNode(5222211):27 points - 4 fields,
       OctNode(4766):27 points - 4 fields,
       OctNode(765):27 points - 4 fields,
       OctNode(460):27 points - 4 fields,
       OctNode(43333146):26 points - 4 fields,
       OctNode(43333113):26 points - 4 fields,
       OctNode(56737):26 points - 4 fields,
       OctNode(4764):26 points - 4 fields,
       OctNode(7000005150):25 points - 4 fields,
       OctNode(52222057):25 points - 4 fields,
       OctNode(43333275):25 points - 4 fields,
       OctNode(43333147):25 points - 4 fields,
       OctNode(5222020):25 points - 4 fields,
       OctNode(647):25 points - 4 fields,
       OctNode(105):25 points - 4 fields,
       OctNode(522223422):24 points - 4 fields,
       OctNode(522223420):24 points - 4 fields,
       OctNode(52222267):24 points - 4 fields,
       OctNode(52222064):24 points - 4 fields,
       OctNode(52222047):24 points - 4 fields,
       OctNode(4333131):24 points - 4 fields,
       OctNode(7000004356):23 points - 4 fi

In [55]:
for node in octree.node_list[::-1]:
    if hasattr(node,'buffer_coordss'): node.buffer_size = len(node.buffer_coordss)
    else: node.buffer_size = 0

In [30]:
from operator import attrgetter,methodcaller
for node in node_list:
    if hasattr(node,'buffer_coordss') is not None: print(node)

OctNode(6111115147):1 points - 4 fields
OctNode(5222226357):1 points - 4 fields
OctNode(5222226256):1 points - 4 fields
OctNode(5222226255):1 points - 4 fields
OctNode(4333337355):1 points - 4 fields
OctNode(522223667):1 points - 4 fields
OctNode(3253):1 points - 4 fields
OctNode(3013):1 points - 4 fields
OctNode(1071):1 points - 4 fields
OctNode(5222226377):2 points - 4 fields
OctNode(5222226355):2 points - 4 fields
OctNode(5222226257):2 points - 4 fields
OctNode(5222226254):2 points - 4 fields
OctNode(433330373):2 points - 4 fields
OctNode(6256):2 points - 4 fields
OctNode(7000006147):3 points - 4 fields
OctNode(7000004277):3 points - 4 fields
OctNode(5222226374):3 points - 4 fields
OctNode(5222226356):3 points - 4 fields
OctNode(43333231):3 points - 4 fields
OctNode(43333122):3 points - 4 fields
OctNode(2402):3 points - 4 fields
OctNode(236):3 points - 4 fields
OctNode(7000006144):4 points - 4 fields
OctNode(7000004275):4 points - 4 fields
OctNode(5222226376):4 points - 4 fields
Oct

In [8]:
foo = octree.node_list[0]
foo.fields

{'Masses': 0.005686212549335323,
 'com_x': 0.11179892445819087,
 'com_y': 0.8268328181610287,
 'com_z': -0.38125960567559114}

In [14]:
np.sum(masses[:,None]*coords,axis=0)

array([ 0.05490658,  0.40416597, -0.18240251])

In [16]:
masses.size

483

In [11]:
coords = np.array(octree.node_list[0].raw_coords)
masses = np.array(foo.raw_fields)[:,0]